In [ ]:
def matchids(id_current,id_child_current,id_next,id_child_next,id_generation_next):   
  ind=np.array(0)
  for i in range(len(id_current)):
    match=np.where((id_next==id_current[i])&(id_child_next==id_child_current[i])&(id_generation_next<30))
    print("\nFound the id",id_next[match],"at the index",match[0],"in this snapshot")
    ind=np.append(ind,match)
  ind_tracked_id_next=ind[1:len(ind)] 
  print("\nThese are the indices of the ids that matched in current snapshot\n",ind_tracked_id_next)
  return ind_tracked_id_next  

tracked_data_all_clusters_each_snap={}     #
for i in range(total_snaps):               
  part=gizmo.io.Read.read_snapshots(['star'],'snapshot_index', snap, simulation_name=simname, simulation_directory=simdir, assign_hosts=True, assign_hosts_rotation=True)               #snap is the snapshot number here that changes everytime the loop iterates. It starts with sanpshot_start
  
  id=part['star'].prop('id')
  id_child=part['star'].prop('id.child')
  id_generation=part['star'].prop('id.generation')
  age=part['star'].prop('age')
  x=part['star'].prop('host.distance.principal')[:,0] #x component of the position of all stars 
  y=part['star'].prop('host.distance.principal')[:,1] #y component of the position of all stars
  z=part['star'].prop('host.distance.principal')[:,2] #z component of the position of all stars
  n=len(x) # counting the total no. of star particles
  vx=part['star'].prop('host.velocity.principal')[:,0]
  vy=part['star'].prop('host.velocity.principal')[:,1]
  vz=part['star'].prop('host.velocity.principal')[:,2]
  mass=part['star']['mass'] #mass of all stars in snapshot 691
  # compute 3-D velocity in cylindrical coordinates
  # first value is along the major axes (positive definite)
  # secod value is azimuthal velocity in the plane of the disk (positive definite)
  # third value is vertical velocity wrt the disk (signed)
  vR_cyl=part['star'].prop('host.velocity.principal.cylindrical')[:,0]
  vphi_cyl=part['star'].prop('host.velocity.principal.cylindrical')[:,1]
  vz_cyl=part['star'].prop('host.velocity.principal.cylindrical')[:,2]
  print("\n#########\n############\nLoaded id,id_child,age,x,y,z,vx,vy,vz,mass, cylindrical velocities and number of particles for snapshot no.",snap)
  print("Total no of particles in this shapshot no.",snap,"is",n)
  
  
  
  
  
  ################################################################
  ################################################################
  ### Now matching the IDs of this snapshot with the ids of all clusters
  test_cluster=1                       #This is the test cluster to begin with. It resets to 1 at each snapshot 
  tracked_data_all_clusters={}         #This dictionary keeps the tracked information for all star clusters in given snapshot. It also resets with snapshots.
  for j in range(total_clusters):
    tracked_data={}                    #This dictionary holds the tracked information for a cluster temporarily until it is pushed to tracked_data_all_clusters
    print("\n\nNow Matching the ids of the snapshot",snap," with the cluster group id",test_cluster)
    id_test_cluster=import_cluster[test_cluster]["id"]
    id_child_test_cluster=import_cluster[test_cluster]["id_children"]
    sortind=np.argsort(id_test_cluster)
    id_test_cluster_sorted=id_test_cluster[sortind]
    id_child_test_cluster_sorted=id_child_test_cluster[sortind]
    print("The total no. of stars in cluster id ",test_cluster," is",len(id_test_cluster_sorted))
    print("Sorted ids of this cluster is",id_test_cluster_sorted)
    ind_tracked=matchids(id_test_cluster_sorted,id_child_test_cluster_sorted,id,id_child,id_generation)
    
    ################################################################
    ################################################################
    #Now finding the postion, velocities, age and mass of the tracked stars in each snapshots
    age_tracked=age[ind_tracked]
    x_tracked=x[ind_tracked]
    y_tracked=y[ind_tracked]
    z_tracked=z[ind_tracked]
    vx_tracked=vx[ind_tracked]
    vy_tracked=vy[ind_tracked]
    vz_tracked=vz[ind_tracked]
    mass_tracked=mass[ind_tracked]
    vR_cyl_tracked=vR_cyl[ind_tracked]
    vphi_cyl_tracked=vphi_cyl[ind_tracked]
    vz_cyl_tracked=vz_cyl[ind_tracked]
    
    
    ################################################################
    ################################################################
    #Now calculating center of mass and realted properties  
    xcm=distance_functions.cm(x_tracked,mass_tracked)
    ycm=distance_functions.cm(y_tracked,mass_tracked)
    zcm=distance_functions.cm(z_tracked,mass_tracked)
    delta_rxyz=distance_functions.dr(x_tracked,y_tracked,z_tracked,mass_tracked)
    rmax=distance_functions.drmax(x_tracked,y_tracked,z_tracked,mass_tracked)
    ymax=(ycm+1.1*rmax)
    ymin=(ycm-1.1*rmax)
    xmax=(xcm+1.1*rmax)
    xmin=(xcm-1.1*rmax)
    avg_delta_rxyz=np.mean(np.absolute(delta_rxyz))
  
  
    ################################################################
    ################################################################
    #Now lets write our tracked data from each snapshots to a file
    tracked_data={"ind_tracked":ind_tracked,"age_tracked":age_tracked,"x_tracked":x_tracked,"y_tracked":y_tracked,"z_tracked":z_tracked,
    "vx_tracked":vx_tracked,"vy_tracked":vy_tracked,"vz_tracked":vz_tracked,
    "mass_tracked":mass_tracked,"xcm":xcm,"ycm":ycm,"zcm":zcm,"delta_rxyz":delta_rxyz,"rmax":rmax,
    "ymax":ymax,"ymin":ymin,"xmax":xmax,"xmin":xmin,"avg_delta_rxyz":avg_delta_rxyz,
    "vR_cyl_tracked":vR_cyl_tracked,"vphi_cyl_tracked":vphi_cyl_tracked,"vz_cyl_tracked":vz_cyl_tracked}
    